In [1]:
import numpy as np 
import pandas as pd 
import statsmodels.api as sm

In [3]:
df = sm.datasets.fair.load_pandas().data

In [5]:
df

,rate_marriage,age,yrs_married,children,religious,educ,occupation,occupation_husb,affairs
0,3.0,32.0,9.0,3.0,3.0,17.0,2.0,5.0,0.111111
1,3.0,27.0,13.0,3.0,1.0,14.0,3.0,4.0,3.230769
2,4.0,22.0,2.5,0.0,1.0,16.0,3.0,5.0,1.400000
3,4.0,37.0,16.5,4.0,3.0,16.0,5.0,5.0,0.727273
4,5.0,27.0,9.0,1.0,1.0,14.0,3.0,4.0,4.666666
...,...,...,...,...,...,...,...,...,...
6361,5.0,32.0,13.0,2.0,3.0,17.0,4.0,3.0,0.000000
6362,4.0,32.0,13.0,1.0,1.0,16.0,5.0,5.0,0.000000
6363,5.0,22.0,2.5,0.0,2.0,14.0,3.0,1.0,0.000000
6364,5.0,32.0,6.0,1.0,3.0,14.0,3.0,4.0,0.000000


In [7]:
def had_affair(x):
    if x != 0:
        return 1
    else:
        return 0

In [9]:
df['had_affair'] = df['affairs'].apply(had_affair)

In [13]:
df['had_affair'].value_counts()

had_affair
0    4313
1    2053
Name: count, dtype: int64

In [15]:
df.groupby('had_affair').mean()

,rate_marriage,age,yrs_married,children,religious,educ,occupation,occupation_husb,affairs
had_affair,,,,,,,,,
0,4.329701,28.390679,7.989335,1.238813,2.504521,14.322977,3.405286,3.833758,0.000000
1,3.647345,30.537019,11.152460,1.728933,2.261568,13.972236,3.463712,3.884559,2.187243


In [23]:
occ_dummies = pd.get_dummies(df['occupation']).astype(int)
hus_dummies = pd.get_dummies(df['occupation_husb']).astype(int)

In [37]:
# Create column names for the new DataFrames
occ_dummies.columns = ['occ1','occ2','occ3', 'occ4', 'occ5', 'occ6']
hus_dummies.columns = ['hocc1','hocc2','hocc3', 'hocc4', 'hocc5', 'hocc6']

In [27]:
# Set X as new DataFrame without the occupation columns or the Y target
X = df.drop(['affairs', 'occupation', 'occupation_husb', 'had_affair'], axis=1)

In [43]:
# Concat the dummy DataFrames Together
dummies = pd.concat([occ_dummies, hus_dummies], axis=1)
dummies = dummies.drop(['occ1', 'hocc1'],axis=1)

In [45]:
# Now Concat the X DataFrame with the dummy variables

X = pd.concat([X, dummies], axis=1)


In [51]:
# Set Y as Target class, Had Affair
y = df['had_affair']


In [6]:
# Dropping one column of each dummy variable set to avoid multicollinearity


# Drop affairs column so Y target makes sense



In [53]:
# This adds a column of 1's to the dataframe. 
# The model will not run without, but if 
# it could every model would try to pass through the origin
logit_model = sm.Logit(y, sm.add_constant(X))

In [55]:
result = logit_model.fit()

Optimization terminated successfully.
         Current function value: 0.542911
         Iterations 6


In [57]:
#result of preliminary run
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:             had_affair   No. Observations:                 6366
Model:                          Logit   Df Residuals:                     6349
Method:                           MLE   Df Model:                           16
Date:                Mon, 06 Oct 2025   Pseudo R-squ.:                  0.1365
Time:                        14:06:36   Log-Likelihood:                -3456.2
converged:                       True   LL-Null:                       -4002.5
Covariance Type:            nonrobust   LLR p-value:                1.534e-222
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const             2.9708      0.572      5.192      0.000       1.849       4.092
rate_marriage    -0.7102      0.031    -22.560      0.000      -0.772      -0.649
age              -0.0613      0.010     -5.936      0.000      -0.082      -0.041
yrs_married       0.1080      0.011      9.836      0.000       0.086       0.129
children          0.0156      0.032      0.488      0.625      -0.047       0.078
religious        -0.3754      0.035    -10.766      0.000      -0.444      -0.307
educ             -0.0017      0.017     -0.099      0.921      -0.036       0.032
occ2              0.3902      0.448      0.872      0.383      -0.487       1.267
occ3              0.7027      0.441      1.592      0.111      -0.163       1.568
occ4              0.4714      0.443      1.065      0.287      -0.396       1.339
occ5              1.0542      0.447      2.360      0.018       0.179       1.930
occ6              1.1080      0.494      2.242      0.025       0.139       2.077
hocc2             0.1704      0.186      0.916      0.360      -0.194       0.535
hocc3             0.2842      0.202      1.406      0.160      -0.112       0.680
hocc4             0.1428      0.181      0.789      0.430      -0.212       0.498
hocc5             0.1723      0.183      0.944      0.345      -0.186       0.530
hocc6             0.1828      0.204      0.897      0.369      -0.216       0.582
=================================================================================
"""

In [61]:
X = X.drop(['children','educ', 'occ2', 'occ3', 'occ4', 'hocc2','hocc3', 'hocc4', 'hocc5', 'hocc6'], axis=1)

In [63]:
X.head()

,rate_marriage,age,yrs_married,religious,occ5,occ6
0,3.0,32.0,9.0,3.0,0,0
1,3.0,27.0,13.0,1.0,0,0
2,4.0,22.0,2.5,1.0,0,0
3,4.0,37.0,16.5,3.0,1,0
4,5.0,27.0,9.0,1.0,0,0


In [65]:
log_model_2 = sm.Logit(y, sm.add_constant(X))

In [67]:
result2 = log_model_2.fit()

Optimization terminated successfully.
         Current function value: 0.544657
         Iterations 6


In [69]:
result2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:             had_affair   No. Observations:                 6366
Model:                          Logit   Df Residuals:                     6359
Method:                           MLE   Df Model:                            6
Date:                Mon, 06 Oct 2025   Pseudo R-squ.:                  0.1337
Time:                        14:12:28   Log-Likelihood:                -3467.3
converged:                       True   LL-Null:                       -4002.5
Covariance Type:            nonrobust   LLR p-value:                5.057e-228
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const             3.7516      0.262     14.317      0.000       3.238       4.265
rate_marriage    -0.7166      0.031    -22.880      0.000      -0.778      -0.655
age              -0.0632      0.010     -6.384      0.000      -0.083      -0.044
yrs_married       0.1127      0.009     12.028      0.000       0.094       0.131
religious        -0.3750      0.035    -10.831      0.000      -0.443      -0.307
occ5              0.4731      0.088      5.398      0.000       0.301       0.645
occ6              0.5271      0.222      2.377      0.017       0.093       0.962
=================================================================================
"""

In [71]:
preds = result2.pred_table()

In [79]:
confusion_matrix = pd.DataFrame(preds, index= ['did not', 'did'], columns = ['pred not', 'pred did'])
confusion_matrix

,pred not,pred did
did not,3881.0,432.0
did,1326.0,727.0


In [81]:
TP = confusion_matrix.loc['did', 'pred did']
FP = confusion_matrix.loc['did not', 'pred did']
TN = confusion_matrix.loc['did not', 'pred not']
FN = confusion_matrix.loc['did', 'pred not']

In [83]:
TPR=(float(TP) / (TP + FN))
TPN=(float(TN) / (TN + FP)) 
PPV=(float(TP) / (TP + FP)) 
NPV=(float(TN) / (TN + FN)) 
FNR=(float(FN) / (FN + TP))
FPR=(float(FP) / (FP + TN))
FDR=(float(FP) / (FP + TP))
FOR=(float(FN) / (FN + TN))
TS=(float(TP) / (TP+FN + FP))
ACC=(float(TP+TN) / (TP+FP+FN + TN))  #print((TP + TN) / float(len(y_test)))

print (f"sensitivity, recall, hit rate, or true positive rate (TPR): {TPR:.3f} (# positives correctly identified)")
print (f"specificity, selectivity or true negative rate (TNR): {TPN:.3f}")
print (f"precision or positive predictive value (PPV): {PPV:.3f} (rate of correct positive predictions)")
print (f"negative predictive value (NPV): {NPV:.3f}")
print (f"miss rate or false negative rate (FNR): {FNR:.3f}")
print (f"fall-out or false positive rate (FPR): {FPR:.3f}")
print (f"false discovery rate (FDR): {FDR:.3f}")
print (f"false omission rate (FOR): {FOR:.3f}")
print("")
print (f"accuracy (ACC): {ACC:.3f} (really only useful if classes are equally represented)")


sensitivity, recall, hit rate, or true positive rate (TPR): 0.354 (# positives correctly identified)
specificity, selectivity or true negative rate (TNR): 0.900
precision or positive predictive value (PPV): 0.627 (rate of correct positive predictions)
negative predictive value (NPV): 0.745
miss rate or false negative rate (FNR): 0.646
fall-out or false positive rate (FPR): 0.100
false discovery rate (FDR): 0.373
false omission rate (FOR): 0.255

accuracy (ACC): 0.724 (really only useful if classes are equally represented)
